Deep Learning
=============


Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [3]:
# Check Python version and pip installed modules
import sys
import pip
installed_packages = pip.get_installed_distributions()
installed_packages_list = sorted(["%s==%s" % (i.key, i.version)
     for i in installed_packages])
print (sys.version)
print (installed_packages_list)

3.5.1 |Anaconda custom (x86_64)| (default, Dec  7 2015, 11:24:55) 
[GCC 4.2.1 (Apple Inc. build 5577)]
['alabaster==0.7.7', 'anaconda-client==1.4.0', 'anaconda-navigator==1.1.0', 'appnope==0.1.0', 'appscript==1.0.1', 'argcomplete==1.0.0', 'astropy==1.1.2', 'babel==2.2.0', 'beautifulsoup4==4.4.1', 'bitarray==0.8.1', 'blaze==0.9.1', 'bokeh==0.11.1', 'boto==2.39.0', 'bottleneck==1.0.0', 'cffi==1.5.2', 'chest==0.2.3', 'cloudpickle==0.1.1', 'clyent==1.2.1', 'colorama==0.3.7', 'conda-build==0+unknown', 'conda-env==2.5.0a0', 'conda-manager==0.3.1', 'conda==4.1.3', 'configobj==5.0.6', 'cryptography==1.3', 'cycler==0.10.0', 'cython==0.23.4', 'cytoolz==0.7.5', 'dask==0.8.1', 'datashape==0.5.1', 'decorator==4.0.9', 'dill==0.2.4', 'docutils==0.12', 'dynd==0.7.3.dev1', 'et-xmlfile==1.0.1', 'fastcache==1.0.2', 'flask-cors==2.1.2', 'flask==0.10.1', 'gevent==1.1.0', 'greenlet==0.4.9', 'gspread==0.3.0', 'h5py==2.5.0', 'heapdict==1.0.0', 'httplib2==0.9.2', 'idna==2.0', 'ipykernel==4.3.1', 'ipython-genut

In [4]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [5]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (52909, 28, 28) (52909,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [6]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (52909, 784) (52909, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [7]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

---
Logistic model
---

In [30]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000
beta = 0.001
graph = tf.Graph()
with graph.as_default():

  # Input data.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)

  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  loss = tf.reduce_mean(loss + beta * tf.nn.l2_loss(weights) ) #L2 Regularization
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [31]:
num_steps = 801

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 21.711681
Training accuracy: 10.8%
Validation accuracy: 12.2%
Loss at step 100: 4.903074
Training accuracy: 70.8%
Validation accuracy: 70.0%
Loss at step 200: 4.009766
Training accuracy: 74.0%
Validation accuracy: 72.6%
Loss at step 300: 3.425276
Training accuracy: 75.3%
Validation accuracy: 73.8%
Loss at step 400: 2.983362
Training accuracy: 76.7%
Validation accuracy: 74.6%
Loss at step 500: 2.630995
Training accuracy: 77.7%
Validation accuracy: 74.9%
Loss at step 600: 2.341271
Training accuracy: 78.7%
Validation accuracy: 75.4%
Loss at step 700: 2.099028
Training accuracy: 79.5%
Validation accuracy: 75.8%
Loss at step 800: 1.894294
Training accuracy: 80.3%
Validation accuracy: 76.2%
Test accuracy: 84.1%


---
Neural Network model
---

In [34]:
batch_size = 128
beta = 0.001
hidden_nodes_size = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes_size]))
  biases = tf.Variable(tf.zeros([hidden_nodes_size]))
  
  weights_h = tf.Variable(
    tf.truncated_normal([hidden_nodes_size, num_labels]))
  biases_h = tf.Variable(tf.zeros([num_labels]))

  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases #1 first layer
  relu = tf.nn.relu(logits) # hidden layer relu
  logits2 = tf.matmul(relu, weights_h) + biases_h #2 second layer
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits2, tf_train_labels)) #loss function
  loss = tf.reduce_mean(loss + beta * tf.nn.l2_loss(weights_h) ) #L2 Regularization
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits2)
  # Both valid and test are the same as our training computation, but instead I chose to nest them for brevity
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights) + biases), weights_h) + biases_h)
  test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights) + biases), weights_h) + biases_h)

In [35]:
#Unchanged from example prior
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 370.678192
Minibatch accuracy: 8.6%
Validation accuracy: 34.1%
Minibatch loss at step 500: 11.336382
Minibatch accuracy: 82.0%
Validation accuracy: 77.7%
Minibatch loss at step 1000: 6.359244
Minibatch accuracy: 83.6%
Validation accuracy: 81.4%
Minibatch loss at step 1500: 2.937435
Minibatch accuracy: 83.6%
Validation accuracy: 81.4%
Minibatch loss at step 2000: 1.774760
Minibatch accuracy: 90.6%
Validation accuracy: 80.6%
Minibatch loss at step 2500: 2.551410
Minibatch accuracy: 85.2%
Validation accuracy: 82.0%
Minibatch loss at step 3000: 2.216882
Minibatch accuracy: 93.0%
Validation accuracy: 82.1%
Test accuracy: 89.4%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

Neural Network Model, 1 hidden layer, L2 Regularized. Training data limited to 1000 vectors

In [32]:
batch_size = 128
beta = 0.001
hidden_nodes_size = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes_size]))
  biases = tf.Variable(tf.zeros([hidden_nodes_size]))
  
  weights_h = tf.Variable(
    tf.truncated_normal([hidden_nodes_size, num_labels]))
  biases_h = tf.Variable(tf.zeros([num_labels]))

  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases #1 first layer
  relu = tf.nn.relu(logits) # hidden layer relu
  logits2 = tf.matmul(relu, weights_h) + biases_h #2 second layer
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits2, tf_train_labels)) #loss function
  loss = tf.reduce_mean(loss + beta * tf.nn.l2_loss(weights) ) #L2 Regularization
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits2)
  # Both valid and test are the same as our training computation, but instead I chose to nest them for brevity
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights) + biases), weights_h) + biases_h)
  test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights) + biases), weights_h) + biases_h)

In [36]:
#Unchanged from example prior
num_steps = 3001
restricted_dataset = train_dataset[:1000]
restricted_labels = train_labels[:1000]
with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (restricted_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = restricted_dataset[offset:(offset + batch_size), :]
    batch_labels = restricted_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 477.458832
Minibatch accuracy: 3.9%
Validation accuracy: 18.0%
Minibatch loss at step 500: 2.839514
Minibatch accuracy: 100.0%
Validation accuracy: 76.8%
Minibatch loss at step 1000: 1.722039
Minibatch accuracy: 100.0%
Validation accuracy: 76.8%
Minibatch loss at step 1500: 1.044341
Minibatch accuracy: 100.0%
Validation accuracy: 76.8%
Minibatch loss at step 2000: 0.633354
Minibatch accuracy: 100.0%
Validation accuracy: 76.8%
Minibatch loss at step 2500: 0.384107
Minibatch accuracy: 100.0%
Validation accuracy: 76.8%
Minibatch loss at step 3000: 0.232947
Minibatch accuracy: 100.0%
Validation accuracy: 76.8%
Test accuracy: 83.8%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [71]:
batch_size = 128
beta = 0.001
hidden_nodes = 1024
graph = tf.Graph()
with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable( tf.truncated_normal([hidden_nodes, num_labels])) 
  biases = tf.Variable(tf.zeros([num_labels]))
    
  # Hidden Layer Variables and layer operation
  hidden_weights = tf.Variable( tf.truncated_normal([image_size * image_size, hidden_nodes]) )
  hidden_biases = tf.Variable( tf.zeros([hidden_nodes]))
  hidden_layer = tf.nn.relu( tf.matmul( tf_train_dataset, hidden_weights) + hidden_biases)
  keep_prob = tf.placeholder("float") #Weird, need to declare it beforehand
  hidden_layer_drop = tf.nn.dropout(hidden_layer, keep_prob) #Dropout added

  # Training computation.
  logits = tf.matmul(hidden_layer_drop, weights) + biases #1st Layer + Hidden Layer + Drop
  loss = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels) ) #2nd Layer
  loss = tf.reduce_mean( loss + beta * tf.nn.l2_loss(weights) ) # L2 Regularization

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

  # Predictions for the training, validation, and test data.
  # No dropout for predictions
  train_prediction = tf.nn.softmax(logits)
  valid_relu = tf.nn.relu(  tf.matmul(tf_valid_dataset, hidden_weights) + hidden_biases)
  valid_prediction = tf.nn.softmax( tf.matmul(valid_relu, weights) + biases) 

  test_relu = tf.nn.relu( tf.matmul( tf_test_dataset, hidden_weights) + hidden_biases) #Note that this is hidden layer
    #sans the drop
  test_prediction = tf.nn.softmax(tf.matmul(test_relu, weights) + biases)

---
Test with the regular set

In [72]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]

        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 0.5}
        _, l, predictions = session.run( [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
          print("Minibatch loss at step %d: %f" % (step, l))
          print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
          print("Validation accuracy: %.1f%%" % accuracy( valid_prediction.eval(), valid_labels) )
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 485.313446
Minibatch accuracy: 7.8%
Validation accuracy: 33.6%
Minibatch loss at step 500: 24.955059
Minibatch accuracy: 80.5%
Validation accuracy: 79.6%
Minibatch loss at step 1000: 16.731840
Minibatch accuracy: 72.7%
Validation accuracy: 80.1%
Minibatch loss at step 1500: 7.332307
Minibatch accuracy: 76.6%
Validation accuracy: 80.4%
Minibatch loss at step 2000: 4.546242
Minibatch accuracy: 79.7%
Validation accuracy: 79.5%
Minibatch loss at step 2500: 5.715791
Minibatch accuracy: 71.1%
Validation accuracy: 80.9%
Minibatch loss at step 3000: 4.375486
Minibatch accuracy: 84.4%
Validation accuracy: 80.7%
Test accuracy: 87.9%


---
Now, with the restricted set.

In [74]:
#Unchanged from example prior
num_steps = 3001
restricted_dataset = train_dataset[:1000]
restricted_labels = train_labels[:1000]
with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        offset = (step * batch_size) % (restricted_labels.shape[0] - batch_size)
        
        batch_data = restricted_dataset[offset:(offset + batch_size), :]
        batch_labels = restricted_labels[offset:(offset + batch_size), :]
        
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 0.5}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 441.794464
Minibatch accuracy: 12.5%
Validation accuracy: 14.0%
Minibatch loss at step 500: 3.587610
Minibatch accuracy: 99.2%
Validation accuracy: 79.0%
Minibatch loss at step 1000: 2.111068
Minibatch accuracy: 100.0%
Validation accuracy: 78.4%
Minibatch loss at step 1500: 1.338227
Minibatch accuracy: 100.0%
Validation accuracy: 78.7%
Minibatch loss at step 2000: 0.875609
Minibatch accuracy: 100.0%
Validation accuracy: 78.1%
Minibatch loss at step 2500: 0.578004
Minibatch accuracy: 100.0%
Validation accuracy: 78.6%
Minibatch loss at step 3000: 0.417990
Minibatch accuracy: 100.0%
Validation accuracy: 77.6%
Test accuracy: 85.2%


Both our test accuracy and validation accuracy improve with the dropout, despite overfitting.

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [102]:
batch_size = 128
beta = 0.001
hidden_nodes = 1024
graph = tf.Graph()
with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable( tf.truncated_normal([hidden_nodes, num_labels])) 
  biases = tf.Variable(tf.zeros([num_labels]))
    
  # Hidden Layer Variables and layer operation
  hidden_weights = tf.Variable( tf.truncated_normal([image_size * image_size, hidden_nodes]) )
  hidden_biases = tf.Variable( tf.zeros([hidden_nodes]))
  hidden_layer = tf.nn.relu( tf.matmul( tf_train_dataset, hidden_weights) + hidden_biases)
  keep_prob = tf.placeholder("float") #Weird, need to declare it beforehand
  hidden_layer_drop = tf.nn.dropout(hidden_layer, keep_prob) #Dropout added

  # Training computation.
  logits = tf.matmul(hidden_layer_drop, weights) + biases #1st Layer + Hidden Layer + Drop
  loss = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels) ) #2nd Layer
  loss = tf.reduce_mean( loss + beta * tf.nn.l2_loss(weights) ) # L2 Regularization

  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.1, global_step, 1000, 0.852)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

  # Predictions for the training, validation, and test data.
  # No dropout for predictions
  train_prediction = tf.nn.softmax(logits)
  valid_relu = tf.nn.relu(  tf.matmul(tf_valid_dataset, hidden_weights) + hidden_biases)
  valid_prediction = tf.nn.softmax( tf.matmul(valid_relu, weights) + biases) 

  test_relu = tf.nn.relu( tf.matmul( tf_test_dataset, hidden_weights) + hidden_biases) #Note that this is hidden layer
    #sans the drop
  test_prediction = tf.nn.softmax(tf.matmul(test_relu, weights) + biases)

In [103]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]

        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 0.5}
        _, l, predictions = session.run( [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
          print("Minibatch loss at step %d: %f" % (step, l))
          print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
          print("Validation accuracy: %.1f%%" % accuracy( valid_prediction.eval(), valid_labels) )
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 449.634979
Minibatch accuracy: 12.5%
Validation accuracy: 25.1%
Minibatch loss at step 500: 27.114326
Minibatch accuracy: 79.7%
Validation accuracy: 81.7%
Minibatch loss at step 1000: 15.289356
Minibatch accuracy: 78.1%
Validation accuracy: 83.2%
Minibatch loss at step 1500: 12.765326
Minibatch accuracy: 74.2%
Validation accuracy: 82.2%
Minibatch loss at step 2000: 4.655536
Minibatch accuracy: 85.9%
Validation accuracy: 83.6%
Minibatch loss at step 2500: 5.941658
Minibatch accuracy: 76.6%
Validation accuracy: 83.2%
Minibatch loss at step 3000: 5.599965
Minibatch accuracy: 79.7%
Validation accuracy: 83.0%
Test accuracy: 89.6%
